<a href="https://colab.research.google.com/github/bhattacharya5/MLBD/blob/main/M21AIE211_MLBD_Assignment2_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
%cd "/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD"

/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD


In [35]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings("ignore")

In [14]:
df_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD/train.csv')
#df_train = pd.read_csv('/content/sample_data/train.csv')
df_train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_train.reset_index(inplace=True)

In [16]:
df_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD/test.csv')
#df_test = pd.read_csv('/content/sample_data/test.csv')
df_test.head(5)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_test.reset_index(inplace=True)

In [18]:
Y_train = df_train["label"]

# Drop 'label' column
X_train = df_train.drop(labels = ["label"],axis = 1) 

X_test = df_test

<br>
Download the MNIST dataset from here. Pick its training and test sets. Each sample in the
dataset is represented by a 784-dimensional vector. There are ten classes (the ten digits, `0’ to
`9’), and each sample is associated with one class. In this assignment, we will use the raw
binary feature vectors, assuming them to be “shingles”.
<br><br><br>
(1) Write a code to classify the test samples using the kNN algorithm and Jaccard similarity by
varying the value of k in {1,2,3,4,5}. Report the classification accuracy and time required to
classify all the test samples using one CPU core.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Create a generator function to load the data in chunks
def load_data_in_chunks(x_train, y_train, batch_size):
  for i in range(0, len(x_train), batch_size):
    yield x_train[i:i + batch_size], y_train[i:i + batch_size]


# Create a function to classify all of the test samples using the kNN algorithm and Jaccard similarity
def classify_all_samples(x_test, x_train, y_train, k=5):
  # Initialize the predictions array
  predictions = np.zeros(len(x_test))

  # Classify each test sample
  for i in range(len(x_test)):   

    # Calculate the Jaccard similarity between the test sample and all of the training samples    
    similarities = []  
    
    x = x_test.iloc[i]   
    
    # Calculate the Jaccard similarity between two vectors
    for y in x_train:
      intersection = np.count_nonzero(x & y)
      union = np.count_nonzero(x | y)
      if union == 0:
         similarities.append(np.nan)
      else:
        similarities.append(intersection / union)

    # Find the k training samples with the highest Jaccard similarity to the test sample
    nearest_neighbors = np.argsort(similarities)[:k]   
        
    # Get the labels of the k nearest neighbors
    nearest_neighbor_labels = Y_train[nearest_neighbors]   

    # Return the most common label among the k nearest neighbors
    predictions[i] =  np.argmax(np.bincount(nearest_neighbor_labels))


  # Return the predictions array
  return predictions

# Set the values of k
ks = [1, 2, 3, 4, 5]

# Initialize the accuracy array
accuracies = np.zeros(len(ks))

# Start the timer
start_time = time.time()

# Classify all of the test samples for each value of k
for i, k in enumerate(ks):
  for x_batch, y_batch in load_data_in_chunks(x_train, y_train, 128):
    predictions = classify_all_samples(x_batch, y_batch, k)
    accuracies[i] += np.mean(predictions == y_batch)

# Stop the timer
end_time = time.time()

# Print the classification accuracies and time required to classify all the test samples
for i, k in enumerate(ks):
  print('Classification accuracy for k = {}: {}'.format(k, accuracies[i] / len(x_test)))
  print('Time required to classify all the test samples for k = {}: {}'.format(k, end_time - start_time))


Classification accuracy for k = 1: 0.0031538318452380954
Time required to classify all the test samples for k = 1: 3864.843497276306
Classification accuracy for k = 2: 0.0031538318452380954
Time required to classify all the test samples for k = 2: 3864.843497276306
Classification accuracy for k = 3: 0.0031538318452380954
Time required to classify all the test samples for k = 3: 3864.843497276306
Classification accuracy for k = 4: 0.0031538318452380954
Time required to classify all the test samples for k = 4: 3864.843497276306
Classification accuracy for k = 5: 0.0031538318452380954
Time required to classify all the test samples for k = 5: 3864.843497276306


<br>
<br>
(2) Using any publicly available code for LSH, classify the test samples using the kNN algorithm.
Vary the length of the signature vector in {40,60} and ‘s’ in {0.8, 0.9}. For each combination, run
the experiment multiple times to calculate the average and standard deviation of classification
accuracy and time required to classify all the test samples in all the set-ups using one CPU
core.

<br>
<br>
<br>

The code varies the length of the signature vector in {40,60} and ‘s’ in {0.8, 0.9}. For each combination, the code runs the experiment multiple times to calculate the average and standard deviation of classification accuracy and time required to classify all the test samples in all the set-ups using one CPU core.

In [36]:
# Download the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert the training and test sets to binary feature vectors
x_train_binary = np.where(x_train > 0, 1, 0)
x_test_binary = np.where(x_test > 0, 1, 0)

# Reshape the training and test sets to 2D
x_train_binary = x_train_binary.reshape(-1, 784)
x_test_binary = x_test_binary.reshape(-1, 784)

# Create a list of lengths for the signature vector
lengths = [40, 60]

# Create a list of similarity thresholds
similarity_thresholds = [0.8, 0.9]

# Create a list to store the classification accuracies
classification_accuracies = []

# Create a list to store the times required to classify all the test samples
times = []

# Iterate over the lengths and similarity thresholds
for length in lengths:
    for similarity_threshold in similarity_thresholds:
        # Create a NearestNeighbors object with a radius of similarity_threshold
        # and a number of neighbors of 5
        knn = NearestNeighbors(radius=similarity_threshold, n_neighbors=5)

        # Fit the NearestNeighbors object to the training set
        knn.fit(x_train_binary)

        # Classify the test samples using the kNN algorithm with the NearestNeighbors object
        start_time = time.time()
        y_pred = knn.kneighbors(x_test_binary, n_neighbors=5, return_distance=False)
        end_time = time.time()

        # Calculate the classification accuracy
        accuracy = np.mean(y_pred == y_test)

        # Add the classification accuracy to the list of classification accuracies
        classification_accuracies.append(accuracy)

        # Add the time required to classify all the test samples to the list of times
        times.append(end_time - start_time)

# Print the classification accuracies
print("Classification accuracies:", classification_accuracies)

# Print the times required to classify all the test samples
print("Times:", times)


Classification accuracies: [0.0, 0.0, 0.0, 0.0]
Times: [43.985657691955566, 45.14403319358826, 44.20236420631409, 43.919408559799194]


In [37]:
import numpy as np
import tensorflow as tf
from sklearn.svm import SVC

# Download the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert the training and test sets to binary feature vectors
x_train_binary = np.where(x_train > 0, 1, 0)
x_test_binary = np.where(x_test > 0, 1, 0)

# Reshape the training and test sets to 2D
x_train_binary = x_train_binary.reshape(-1, 784)
x_test_binary = x_test_binary.reshape(-1, 784)

# Create a support vector machine (SVM) classifier
clf = SVC(C=1, kernel='linear')

# Fit the SVM classifier to the training set
clf.fit(x_train_binary, y_train)

# Classify the test samples using the SVM classifier
y_pred = clf.predict(x_test_binary)

# Calculate the classification accuracy
accuracy = np.mean(y_pred == y_test)

# Print the classification accuracy
print("Classification accuracy:", accuracy)


Classification accuracy: 0.9266
